# Space


In [ ]:
import sys
import os 
import logging
import pandas as pd
from pprint import pprint 
from IPython.display import display, HTML

KEY = '1-WORKSPACE'
WORKSPACE_PATH = os.getcwd().split(KEY)[0]
print(WORKSPACE_PATH); os.chdir(WORKSPACE_PATH)
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='[%(levelname)s:%(asctime)s:(%(filename)s@%(lineno)d %(name)s)]: %(message)s')

SPACE = {
    'DATA_RAW': f'./_Data/0-Data_Raw',
    'DATA_RFT': f'./_Data/1-Data_RFT',
    'DATA_CASE': f'./_Data/2-Data_CASE',
    'DATA_AIDATA': f'./_Data/3-Data_AIDATA',
    'DATA_EXTERNAL': f'./code/external',
    'CODE_FN': f'./code/pipeline', 
}
assert os.path.exists(SPACE['CODE_FN']), f'{SPACE["CODE_FN"]} not found'

print(SPACE['CODE_FN'])
sys.path.append(SPACE['CODE_FN'])

# Step 1: OneCohort Record Base

In [ ]:
# Import CohortName_to_OneCohortArgs from the Cohort module in config.config_record
from config.config_record.Cohort import CohortName_to_OneCohortArgs
# Create a list comprehension to extract all keys from CohortName_to_OneCohortArgs dictionary
CohortNames = [i for i in CohortName_to_OneCohortArgs.keys()]
# Display the CohortNames
CohortNames 

In [ ]:
# create a dictionary to store the arguments for the HumanRecordRecfeat module
CohortName = 'WellDoc2023CVSDeRx'
HumanRecordRecfeat_Args = {
    'P': {
        'P': ['P-DemoCate'], 
        'PHeight': [], 
        'CGM5Min': [], 
        'Weight': [],
    }
}
HumanRecordRecfeat_Args

In [ ]:
# Import the OneCohort_Record_Base class from the recfldtkn.record_base module
from recfldtkn.record_base import OneCohort_Record_Base
# Create an instance of the OneCohort_Record_Base class
onecohort_record_base = OneCohort_Record_Base(CohortName, 
                                              HumanRecordRecfeat_Args,
                                              CohortName_to_OneCohortArgs,
                                              SPACE
                                            )
# Set up the mapping from the feature name to the function that will be used to extract the feature
onecohort_record_base.setup_NameToFn()
# Initialize the NameToObject dictionary
onecohort_record_base.initialize_NameToObject()

In [ ]:
# shows the content of Name_to_HRF from the onecohort_record_base class
onecohort_record_base.Name_to_HRF

# A Human record object 
# A record object of Patient
# A RecFeat (record feature) object named P-DemoCate
# The nested structure (e.g., ('P', 'P', 'P-DemoCate')) could represent a hierarchy of record types or a way to organize features within record types.

In [ ]:
# Retrieve the Record object for CGM5Min from the Name_to_HRF dictionary
record = onecohort_record_base.Name_to_HRF[('P', 'CGM5Min')]

# Access the ds_RecIndex attribute of the record ob
record.ds_RecIndex

In [ ]:
HumanID = 'PID'

ds_RecIndex = record.ds_RecIndex
df_RecIndex = ds_RecIndex.to_pandas().set_index(HumanID)

df_RecIndex

In [ ]:
onecohort_record_base.Name_to_Fn

In [ ]:
onecohort_record_base

In [ ]:
record = onecohort_record_base.Name_to_HRF[('P', 'P')]

df = record.ds_RecAttr.to_pandas()
df

In [ ]:
df['UserTimeZoneOffset'].isna().sum()

# Step 2: ROName Parser

In [ ]:
# Define a string representing a Record Observation Name
ROName = 'hP.rWeight.cBf2M' # previous two month CGM5min data
# Parse_ROName is called to parse this string. The function breaks down ROName
RONameInfo = onecohort_record_base.parse_ROName(ROName)
RONameInfo

In [ ]:
record = onecohort_record_base.Name_to_HRF[('P', 'P')] # HumanRecordFeat
# Access the ds_RecAttr attribute of the record object 
# The ds_RecAttr attribute is a dataset object with features and it contains 69 rows of data.
record.ds_RecAttr
# ds_RecAttr appears to be an actual dataset, not just a list of column names

In [ ]:
# record.ds_RecAttr.to_pandas().head()

In [ ]:
onecohort_record_base.Name_to_HRF[('P', 'Weight')].ds_RecAttr

# Step 3: TriggerName

In [ ]:
# from recfldtkn.case_base.cohort_case_base import CaseSet_Trigger_Fn

###################################### # user 
Trigger = 'WeightEntry'
Trigger_Args = {
    'Trigger': 'WeightEntry',
    'case_id_columns': ["PID", "ObsDT"], 
    'HumanID_list': ['PID'],
    'ObsDT': 'ObsDT',
    'ROName_to_RONameArgs': {
        'hP.rWeight': {
            'attribute_columns': ['PID', 'WeightID', 'DT_s', 'Weight', 'WeightUOMID'],
        },
    },
}
######################################

In [ ]:
# Import 'inspect' module, which provides several useful functions to help get information about live objects
import inspect

###################################### user
def get_CaseTrigger_from_RecordBase(onecohort_record_base, Trigger_Args):

    TriggerName = Trigger_Args['Trigger']
    if TriggerName in onecohort_record_base.TriggerName_to_dfCaseTrigger:
        df_case_raw = onecohort_record_base.TriggerName_to_dfCaseTrigger[TriggerName]
    else:
        ROName_to_RONameArgs = Trigger_Args['ROName_to_RONameArgs']

        ROName_to_RODS = {}
        for ROName, ROName_Args in ROName_to_RONameArgs.items():
            RONameInfo = onecohort_record_base.parse_ROName(ROName)
            # print(RONameInfo)
            HumanName, RecordName = RONameInfo['HumanName'], RONameInfo['RecordName']
            record = onecohort_record_base.Name_to_HRF[(HumanName, RecordName)]

            attribute_columns = ROName_Args['attribute_columns']
            if hasattr(record, 'ds_RecAttr'):
                df_case = record.ds_RecAttr.select_columns(attribute_columns).to_pandas()
            else:
                df_case = record.df_RecAttr

            ROName_to_RODS[ROName] = df_case
        # -------------------- s: deal with ROName = 'hP.rCGM5Min' ------------------
        ROName = 'hP.rWeight'
        df_case_raw = ROName_to_RODS[ROName]
        # --------------------------------------------------------------------------

    # ------------------------------ Determine the ObsDT ------------------------------
    df_case = df_case_raw
    if 'ObsDT' not in df_case.columns:
        df_case['ObsDT'] = df_case['DT_s'] # [df_case_raw['drug_name'] == 'Trulicity']
        df_case = df_case.drop(columns = 'DT_s') # .from_pandas(df_case_filter)
        df_case['ObsDT'] = pd.to_datetime(df_case['ObsDT'].dt.date) + pd.to_timedelta(1, unit = 'd') - pd.to_timedelta(1, unit = 'second')
    else:
        df_case['ObsDT'] = pd.to_datetime(df_case['ObsDT'])
    # --------------------------------------------------------------------------


    # ------------------------------- Update Column Sequence ------------------------
    case_id_columns = Trigger_Args['case_id_columns']
    columns = df_case.columns 
    columns = case_id_columns + [col for col in columns if col not in case_id_columns]
    df_case = df_case[columns].reset_index(drop=True)

    df_case = df_case.groupby(case_id_columns).last().reset_index()
    # -------------------- e: deal with ROName = 'hP.rCGM5Min' ------------------
    return df_case

get_CaseTrigger_from_RecordBase.fn_string = inspect.getsource(get_CaseTrigger_from_RecordBase)
######################################

In [ ]:
from recfldtkn.case_base import CASESET_TRIGGER_PATH
from recfldtkn.base import Base

pypath = os.path.join(SPACE['CODE_FN'], CASESET_TRIGGER_PATH, f'{Trigger}.py')
print(pypath) 


prefix = ['import pandas as pd', 'import numpy as np']
string_variables = [Trigger]
iterative_variables = [Trigger_Args]
fn_variables = [get_CaseTrigger_from_RecordBase]
pycode = Base.convert_variables_to_pystirng(string_variables = string_variables, 
                                       iterative_variables = iterative_variables, 
                                       fn_variables = fn_variables, prefix = prefix)

# print(pypath)
if not os.path.exists(os.path.dirname(pypath)): os.makedirs(os.path.dirname(pypath))
with open(pypath, 'w') as file: file.write(pycode)

In [ ]:
df_case = get_CaseTrigger_from_RecordBase(onecohort_record_base, Trigger_Args)
df_case

In [ ]:
# 3588